***
# **FloodProofs Labs - HMC Training - TimeSeries** 

<img style="float: left; padding-right: 80px; padding-left: 5px;" src="img/logo_hmc.png" width="200px" align=”left” >

In the laboratory of **HMC time-series** we will perform the following points:
   * Configure the libraries and the dependecies of the laboratory;
   * Read the configuration file of the laboratory;
   * Read the static datasets of terrain, river networks and outlet sections;
   * Read the dynamic datasets of the time-series (collections and hydrographs);
   * Plot the position of the analyzed outlet section;
   * Plot the time-series of discharge and the time-series of the hmc average forcings.    


## **Import libraries and dependencies**

In [1]:
%matplotlib inline
%matplotlib widget

# Libraries
from library.lib_jupyter_data_io_json import read_file_settings, read_file_ts_hydrograph
from library.lib_jupyter_data_io_generic import define_file_path, define_file_template, fill_file_template, create_dframe_ts

from library.lib_jupyter_data_geo_ascii import read_data_grid
from library.lib_jupyter_data_geo_shapefile import read_data_section, find_data_section

from library.lib_jupyter_data_io_netcdf import read_file_ts_collections

from library.lib_jupyter_plot_ts import plot_ts_discharge, plot_ts_forcing

# Define configuration file
file_name_settings="fp_labs_analyzer_hmc_timeseries.json"

# Info
print(' ==> Libraries loaded')

 ==> Libraries loaded


### **Configure the flood-proofs laboratory**
- Load the configuration file:

In [2]:
# Read data from settings algorithm file
settings_info = read_file_settings(file_name_settings)

- Define the static and dynamic file paths:

In [3]:
# Define static and dynamic file path(s)
file_path_dset_static = define_file_path(settings_info['source']['static'])
file_path_dset_dynamic = define_file_path(settings_info['source']['dynamic'])
file_path_dset_plot = define_file_path(settings_info['destination']['plot'])

### **Read the static datasets**
- Read the terrain and river networks files in ascii format

In [4]:
# Read terrain datasets
dset_terrain = read_data_grid(file_path_dset_static['terrain'])

In [5]:
# Read river network datasets
darray_river_network = read_data_grid(file_path_dset_static['river_network'], var_limit_min=0, var_limit_max=1)

- Read the sections information in vector format

In [6]:
# Read sections shapefile
dset_section = read_data_section(file_path_dset_static['sections'])

- Define the information of section, domain and time

In [7]:
# Get domain, section and time information
info_section = find_data_section(dset_section, section_name=settings_info['info']['section_name'],
                                 basin_name=settings_info['info']['basin_name'])
info_domain = settings_info['info']['domain_name']
info_time_run = settings_info['time_run']

# Fill dynamic file path(s)
file_template_filled = define_file_template(
    info_time_run, section_name=info_section['section_name'], basin_name=info_section['section_domain'],
    domain_name=info_domain, template_default=settings_info['template'])
file_path_dset_dynamic = fill_file_template(file_path_dset_dynamic,
                                            template_filled=file_template_filled,
                                            template_default=settings_info['template'])
file_path_dset_plot = fill_file_template(file_path_dset_plot,
                                         template_filled=file_template_filled,
                                         template_default=settings_info['template'])
# Info
print(' ==> SectionName: ' + info_section['section_name'] + ' -- BasinName: ' + info_section['section_domain'] + ' -- TimeRun: ' + info_time_run)

 ==> SectionName: Tronto -- BasinName: Tronto -- TimeRun: 2021-01-22 06:00


### **Read the dynamic datasets**
- Read the collections file in netcdf format

In [8]:
# Read the collections file 
dframe_ts_cls, attrs_ts_cls = read_file_ts_collections(file_path_dset_dynamic['time_series_collections'])
# Info
print(' ==> Collections dynamic file ' + file_path_dset_dynamic['time_series_collections'] + '... LOADED')

 ==> Collections dynamic file /home/fabio/fp_labs_datasets/data_dynamic/20210122_06/data_archive/collections/hmc.collections.202101220600.nc... LOADED


- Read the hydrograph file in json format

In [9]:
# Read the hydrograph file
dframe_ts_cls, attrs_cls = read_file_ts_collections(file_path_dset_dynamic['time_series_collections'])
dframe_ts_hydro, attrs_ts = read_file_ts_hydrograph(file_path_dset_dynamic['time_series_hydrograph'])
# Info
print(' ==> Hydrograph dynamic file ' + file_path_dset_dynamic['time_series_hydrograph'] + ' ... LOADED')

 ==> Hydrograph dynamic file /home/fabio/fp_labs_datasets/data_dynamic/20210122_06/data_archive/collections/hydrograph_Tronto_Tronto_202101220600.json ... LOADED


- Select the variable(s) and create the dataframe

In [10]:
# Select output variable(s) to plot time-series
dframe_ts_discharge_obs = create_dframe_ts(dframe_ts_hydro, var_name_in='discharge_observed', var_name_out='discharge_observed')
dframe_ts_discharge_sim = create_dframe_ts(dframe_ts_hydro, var_name_in='discharge_simulated', var_name_out='discharge_simulated')
dframe_ts_rain = create_dframe_ts(dframe_ts_cls, var_name_in='rain', var_name_out='rain')
dframe_ts_sm = create_dframe_ts(dframe_ts_cls, var_name_in='soil_moisture', var_name_out='soil_moisture')

# Select forcing variable(s) to plot time-series
dframe_ts_airt = create_dframe_ts(dframe_ts_cls, var_name_in='air_temperature', var_name_out='air_temperature')
dframe_ts_incrad = create_dframe_ts(dframe_ts_cls, var_name_in='incoming_radiation', var_name_out='incoming_radiation')
dframe_ts_rh = create_dframe_ts(dframe_ts_cls, var_name_in='relative_humidity', var_name_out='relative_humidity')
dframe_ts_wind = create_dframe_ts(dframe_ts_cls, var_name_in='wind', var_name_out='wind')

### **Plot the dynamic datasets in time-series format**
- Create the plot for analyzing the hmc discharges output

In [11]:
file_name_ts = '/home/fabio/Desktop/PyCharm_Workspace/fp-labs-master/hmc/img/test_ts.png'
plot_ts_discharge(file_name_ts, dframe_ts_discharge_sim, attrs_ts,
                  df_discharge_obs=dframe_ts_discharge_obs, df_rain=dframe_ts_rain, df_soil_moisture=dframe_ts_sm)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

- Create the plot for analyzing the hmc forcing datasets

In [12]:
# Plot ts forcing
file_name_ts_forcing = file_path_dset_plot['time_series_forcing']
plot_ts_forcing(file_name_ts_forcing,
                df_rain=dframe_ts_rain, df_airt=dframe_ts_airt, df_incrad=dframe_ts_incrad,
                df_rh=dframe_ts_rh, df_winds=dframe_ts_wind,
                attrs_forcing=attrs_ts)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …